In [2]:
import numpy as np
import pandas as pd

import numba
import iqplot
import colorcet
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

## Calculate p-values
Null hypothesis: Male and female SHERPies have the same number of Twitter followers.

In [20]:
df = pd.read_csv('SHERP_tweeter_data_clean.csv')
df.tail()

,gender,name,display_name,bio,favorites_count,followers_count,list_memberships,tweets_retweets_count,following_count,acct_created,location
255,f,LauraGeggel,Laura Geggel,"b""Im a transplanted Seattleite living in New Y...",5562,2946,100,3886,4985,2/4/2008 19:46,New York
256,f,CatherineDold,Catherine Dold,Freelance writer/editor #health #biopharm #add...,281,1468,125,1829,1687,2/2/2008 18:31,"Boulder, Colorado"
257,m,kerthan,Ker Than,Sci writer with interests spanning anthropolog...,118,902,78,557,842,1/24/2008 16:26,California
258,f,karina4848,karina4848,Editor of Scholastic MATH (an awesome math mag...,0,81,8,385,90,12/14/2007 3:16,"New York, NY"
259,f,lindy2350,Melinda Wenner Moyer,Science journalist. Contributing editor @SciAm...,16180,13900,624,5181,3904,9/4/2007 19:33,"Hudson Valley, NY"


In [27]:
df.shape

(260, 11)

In [30]:
is_high = df.followers_count > 30000

In [31]:
df[is_high]

,gender,name,display_name,bio,favorites_count,followers_count,list_memberships,tweets_retweets_count,following_count,acct_created,location
247,m,ivanoransky,Ivan Oransky,"VP, Editorial, @Medscape. Distinguished Writer...",10,33404,1754,30063,3020,8/15/2008 2:43,"Northampton, MA and New York, NY"


In [21]:
p = iqplot.ecdf(df, q="followers_count", cats="gender")
bokeh.io.show(p)

In [22]:
rg = np.random.default_rng()

# Set up Numpy arrays for convenience (also much better performance)
male = df.loc[df["gender"] == "m", "followers_count"].values
female = df.loc[df["gender"] == "f", "followers_count"].values

# ECDF values for plotting
male_ecdf = np.arange(1, len(male)+1) / len(male)
female_ecdf = np.arange(1, len(female)+1) / len(female)

# Make 100 bootstrap samples and plot them
for _ in range(100):
    bs_male = rg.choice(male_ecdf, size=len(male_ecdf))
    bs_female = rg.choice(female_ecdf, size=len(female_ecdf))

    # Add semitransparent ECDFs to the plot
    p.circle(np.sort(bs_male), male_ecdf, color=colorcet.b_glasbey_category10[0], alpha=0.02)
    p.circle(np.sort(bs_female), female_ecdf, color=colorcet.b_glasbey_category10[1], alpha=0.02)

bokeh.io.show(p)

In [23]:
# Extract NumPy arrays (only use values greater than zero for logs)
inds = (df["gender"] == 'm') | (df["gender"] == 'f')
df = df.loc[inds, :]

male_followers = df.loc[df["gender"] == "m", "followers_count"].values

male_following = df.loc[df["gender"] == "m", "following_count"].values

female_followers = df.loc[df["gender"] == "f", "followers_count"].values

female_following = df.loc[df["gender"] == "f", "following_count"].values

In [13]:
@numba.njit
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=len(data))


@numba.njit
def bivariate_r(x, y):
    """
    Compute plug-in estimate for the bivariate correlation coefficient.
    """
    return (
        np.sum((x - np.mean(x)) * (y - np.mean(y)))
        / np.std(x)
        / np.std(y)
        / np.sqrt(len(x))
        / np.sqrt(len(y))
    )

In [14]:
@numba.njit
def draw_perm_sample(x, y):
    """Generate a permutation sample."""
    concat_data = np.concatenate((x, y))
    np.random.shuffle(concat_data)

    return concat_data[:len(x)], concat_data[len(x):]


def draw_perm_reps(x, y, stat_fun, size=1):
    """Generate array of permuation replicates."""
    return np.array([stat_fun(*draw_perm_sample(x, y)) for _ in range(size)])

In [15]:
# Draw replicates on the difference of means.

@numba.njit
def draw_perm_reps_diff_mean(x, y, size=1):
    """Generate array of permuation replicates."""
    out = np.empty(size)
    for i in range(size):
        x_perm, y_perm = draw_perm_sample(x, y)
        out[i] = np.mean(x_perm) - np.mean(y_perm)

    return out

In [32]:
# Compute test statistic for original data set
diff_mean = np.mean(male_followers) - np.mean(female_followers)

# Draw replicates
perm_reps = draw_perm_reps_diff_mean(male_followers, female_followers, size=10000000)

# Compute p-value
p_val = np.sum(perm_reps >= diff_mean) / len(perm_reps)

print('p-value =', p_val)

p-value = 0.0615868


In [33]:
diff_mean

1129.8847517730496

Another hypothesis: There is no correlation between follower and following ratios.

In [36]:
# Shift data sets
total_mean = np.mean(np.concatenate((male_followers, female_followers)))
male_shift = male_followers - np.mean(male_followers) + total_mean
female_shift = female_followers - np.mean(female_followers) + total_mean

# Plot the ECDFs
df_shift = pd.DataFrame(
    data={
        "gender": ["m"] * len(male_shift)
        + ["f"] * len(female_shift),
        "followers_count": np.concatenate((male_shift, female_shift)),
    }
)
p = iqplot.ecdf(df_shift, q="followers_count", cats="gender")

bokeh.io.show(p)

In [37]:
p = iqplot.ecdf(
    df,
    q="followers_count",
    cats="gender",
    x_axis_label="followers_count",
)

for _ in range(100):
    male_rep = draw_bs_sample(male_shift)
    female_rep = draw_bs_sample(female_shift)
    df_rep = pd.DataFrame(
        data={
            "gender": ["m"] * len(male_rep) + ["f"] * len(female_rep),
            "followers_count": np.concatenate((male_rep, female_rep)),
        }
    )

    p = iqplot.ecdf(
        df_rep,
        q="followers_count",
        cats="gender",
        p=p,
        marker_kwargs=dict(alpha=0.02),
    )

bokeh.io.show(p)

In [38]:
@numba.njit
def draw_bs_reps_diff_mean(x, y, size=1):
    """
    Generate bootstrap replicates with difference of means
    as the test statistic.
    """
    out = np.empty(size)
    for i in range(size):
        out[i] = np.mean(draw_bs_sample(x)) - np.mean(draw_bs_sample(y))

    return out


# Generate samples (10 million again)
bs_reps = draw_bs_reps_diff_mean(male_shift, female_shift, size=10000000)

# Compute p-value
p_val = np.sum(bs_reps >= diff_mean) / len(bs_reps)

print("p-value =", p_val)

p-value = 0.0839088


Accept the null hypothesis. The p-value is the probably of observing the test statistic being at least as extreme as what was measured if the null hypothesis is true.

In [40]:
%load_ext watermark

%watermark -v -p numpy,pandas,numba,altair,bokeh,colorcet,iqplot,jupyterlab

CPython 3.7.9
IPython 7.19.0

numpy 1.19.5
pandas 1.1.5
numba 0.51.2
altair not installed
bokeh 2.2.3
colorcet 2.0.2
iqplot 0.2.0
jupyterlab 2.2.9
